## Future

In [1]:
import scala.concurrent._

In [2]:
Thread.currentThread().getId()

181

In [5]:
val x = Future {
  println("This is my first future")
  println("Thread ID: " + Thread.currentThread().getId)
}

Name: Compile Error
Message: <console>:21: error: Cannot find an implicit ExecutionContext, either import scala.concurrent.ExecutionContext.Implicits.global or use a custom one
         val x = Future {
                        ^
StackTrace: 

In [ ]:
// although these 5 threads will start ALMOST at the same time (maybe still follow 1 --> 5 order at that time)
// there is no guarantee that the result will follow the same order
// each time as they are almost parallel 
import scala.concurrent._
import scala.concurrent.ExecutionContext.Implicits.global

List(1,2,3,4,5).map(x => Future {
    Thread.sleep(2000) // each thread will be slept for 2s and then execute the following code
    println("Item " + x + "ThreadID: " + Thread.currentThread().getId()) 
})

In [6]:
val x = Future {
    Thread.currentThread().getId()
}

Name: Compile Error
Message: <console>:21: error: Cannot find an implicit ExecutionContext, either import scala.concurrent.ExecutionContext.Implicits.global or use a custom one
         val x = Future {
                        ^
StackTrace: 

In [ ]:
import scala.util.{Success, Failure}

val x = Future {
    Thread.currentThread().getId()
}

// onSuccess will only be called when the future is success
// onFailure will only be called when the future is failed
x onComplete { // onComplete is always called, a method for Future
    case Success(res) => { // res is not  result, we do not need to give data type as Scala is inference language, it will automatically check the data type
        println(res)
        println("OnSuccess: " + x + " ThreadID: " + Thread.currentThread().getId())
    }
    case _ => println("Error")
}

In [ ]:
// How to make sure we wait until everything is completed, to do something
import scala.concurrent._
import scala.concurrent.ExecutionContext.Implicits.global

println("Main ThreadID: " + Thread.currentThread().getId())

Future.sequence( // sequence method is to make sure no future action till current all futures are completed 
List(1,2,3,4,5).map(x => Future {
    Thread.sleep(2000) // each thread will be slept for 2s and then execute the following code
    println("Item " + x + "ThreadID: " + Thread.currentThread().getId()) 
})).map(x =>{
    println("Completed. Do something now")
    println("After all done ThreadID: " + Thread.currentThread().getId())
})

In [ ]:
//block until futures are completed
import scala.concurrent._
import scala.concurrent.ExecutionContext.Implicits.global


// await.ready is similar with future.sequence except await.ready is a blocking method while future.sequence is non-blocking method
// so when there is an indepedent code right after the block, for sequence, this code will be executed, while await will not be run
Await.ready(
   Future.sequence(
    List(1,2,3,4,5).map(x => Future {
    Thread.sleep(2000) // each thread will be slept for 2s and then execute the following code
    println("Item " + x + "ThreadID: " + Thread.currentThread().getId()) })), duration.Duration.Inf
)

println("Now ready")
println("After all done ThreadID: " + Thread.currentThread().getId())


In [ ]:
// folding units 
import scala.concurrent._
import scala.concurrent.ExecutionContext.Implicits.global

(Future
    .fold(List(1,2,3,4,5)
        .map(x => Future {
            Thread.sleep(2000) // each thread will be slept for 2s and then execute the following code
            println("Item " + x + "ThreadID: " + Thread.currentThread().getId())
            x
        })
    )(0){(acc,num) => acc + num } // set 0 as the initial start value for folding, then it will be like (((0 + 1)+2)+3)...
)
.map(x=> println(x))

In [ ]:
// reducing results 
import scala.concurrent._
import scala.concurrent.ExecutionContext.Implicits.global

(Future
    .reduce(List(1,2,3,4,5)
        .map(x => Future {
            Thread.sleep(2000) // each thread will be slept for 2s and then execute the following code
            println("Item " + x + "ThreadID: " + Thread.currentThread().getId()) 
            x
        })
    ) {(acc, num) => acc + num } // reduce has the same effect as folding except we do not need to give the start value 
)
.map(x=> println(x))